# Using Subagents and Elastic Agent Builder to Bring Business Context into Code Planning - Setup

Learn how to build a specialized subagent that connects Claude Code to your Elasticsearch data, bringing user analytics and business context into your development workflow.

## Configuration

Create a `.env` file in this directory with your credentials:

```
ELASTICSEARCH_URL=
KIBANA_URL=
API_KEY=
```

In [ ]:
import json
import requests
from dotenv import load_dotenv
import os
from elasticsearch import Elasticsearch
from elasticsearch import helpers

load_dotenv()

ES_URL = os.getenv("ELASTICSEARCH_URL")
KIBANA_URL = os.getenv("KIBANA_URL")
API_KEY = os.getenv("API_KEY")

es_client = Elasticsearch(ES_URL, api_key=API_KEY)

### Method to bulk the data into Elasticsearch

In [ ]:
def build_bulk_actions(documents, index_name):
    for doc in documents:
        yield {"_index": index_name, "_source": doc}

## Knowledge Index

Engineering standards, policies, and roadmap priorities.

In [ ]:
knowledge_mapping = {
    "properties": {
        "title": {"type": "text"},
        "content": {"type": "text"},
        "category": {"type": "keyword"},
        "tags": {"type": "keyword"},
        "source": {"type": "keyword"},
        "created_at": {"type": "date"},
        "updated_at": {"type": "date"},
    }
}

es_client.indices.create(index="knowledge", mappings=knowledge_mapping)

In [ ]:
with open("dataset/knowledge.json", "r") as f:
    knowledge_docs = json.load(f)

success, failed = helpers.bulk(
    es_client,
    build_bulk_actions(knowledge_docs, "knowledge"),
    refresh=True,
)
print(f"{success} documents indexed successfully")

## Error Logs Index

Production error frequency by module with affected users.

In [ ]:
error_logs_mapping = {
    "properties": {
        "module": {"type": "keyword"},
        "error_type": {"type": "keyword"},
        "message": {"type": "text"},
        "severity": {"type": "keyword"},
        "count": {"type": "integer"},
        "user_ids_affected": {"type": "keyword"},
        "timestamp": {"type": "date"},
    }
}

es_client.indices.create(index="error_logs", mappings=error_logs_mapping)

In [ ]:
with open("dataset/error_logs.json", "r") as f:
    error_logs_docs = json.load(f)

success, failed = helpers.bulk(
    es_client,
    build_bulk_actions(error_logs_docs, "error_logs"),
    refresh=True,
)
print(f"{success} documents indexed successfully")

## Support Tickets Index

Customer complaints with urgency markers and tier information.

In [ ]:
support_tickets_mapping = {
    "properties": {
        "ticket_id": {"type": "keyword"},
        "title": {"type": "text"},
        "description": {"type": "text"},
        "related_module": {"type": "keyword"},
        "priority": {"type": "keyword"},
        "status": {"type": "keyword"},
        "customer_tier": {"type": "keyword"},
        "created_at": {"type": "date"},
        "marked_urgent": {"type": "boolean"},
    }
}

es_client.indices.create(index="support_tickets", mappings=support_tickets_mapping)

In [ ]:
with open("dataset/support_tickets.json", "r") as f:
    support_tickets_docs = json.load(f)

success, failed = helpers.bulk(
    es_client,
    build_bulk_actions(support_tickets_docs, "support_tickets"),
    refresh=True,
)
print(f"{success} documents indexed successfully")

## Customer Data Index

Customer tier information (enterprise, pro, free) with MRR.

In [ ]:
customer_data_mapping = {
    "properties": {
        "user_id": {"type": "keyword"},
        "customer_tier": {"type": "keyword"},
        "company_name": {"type": "text"},
        "mrr": {"type": "float"},
        "joined_at": {"type": "date"},
    }
}

es_client.indices.create(index="customer_data", mappings=customer_data_mapping)

In [ ]:
with open("dataset/customer_data.json", "r") as f:
    customer_data_docs = json.load(f)

success, failed = helpers.bulk(
    es_client,
    build_bulk_actions(customer_data_docs, "customer_data"),
    refresh=True,
)
print(f"{success} documents indexed successfully")

## Create Tech Debt Advisor Agent

Retrieval agent with ES|QL and search capabilities via Agent Builder API.

In [ ]:
agent_config = {
    "id": "tech-debt-advisor",
    "name": "Tech Debt Prioritization Agent",
    "description": "I help prioritize technical debt by analyzing error logs, support tickets, customer impact, and aligning with engineering standards and roadmap priorities.",
    "avatar_color": "#BFDBFF",
    "avatar_symbol": "TD",
    "configuration": {
        "instructions": """This agent helps prioritize technical debt items. Use the following indices:

- knowledge: Engineering standards, policies, and roadmap priorities
- error_logs: Production error frequency by module
- support_tickets: Customer complaints and their urgency
- customer_data: Customer tier information (enterprise, pro, free)

When analyzing tech debt:
1. Check error frequency in error_logs
2. Cross-reference affected users with customer_data to understand tier impact
3. Count support tickets and note urgency markers
4. Check knowledge base for relevant policies and Q1 priorities
5. Synthesize findings into prioritized recommendations""",
        "tools": [
            {
                "tool_ids": [
                    "platform.core.search",
                    "platform.core.list_indices",
                    "platform.core.get_index_mapping",
                    "platform.core.get_document_by_id",
                    "platform.core.execute_esql",
                    "platform.core.generate_esql",
                ]
            }
        ],
    },
}

kibana_headers = {
    "Content-Type": "application/json",
    "kbn-xsrf": "true",
    "Authorization": f"ApiKey {API_KEY}",
}

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/agents", json=agent_config, headers=kibana_headers
)

print(f"Status: {response.status_code}")

print(json.dumps(response.json(), indent=2))

## Cleanup

Delete all created indices and the agent.

In [ ]:
indices_to_delete = ["knowledge", "error_logs", "support_tickets", "customer_data"]

for index in indices_to_delete:
    es_client.indices.delete(index=index)
    print(f"Deleted index: {index}")

In [ ]:
response = requests.delete(
    f"{KIBANA_URL}/api/agent_builder/agents/tech-debt-advisor", headers=kibana_headers
)

print(f"Status: {response.status_code}")